<a id='tablecontents'></a>

# PySpark for Data Analysis on Jupyter Notebook - Details for Data Manipulation
<h5>2023, Andrea Paviglianiti</h5>

<hr>

## Table of Contents:

- [Import all dependencies](#section1)
- [Create a Spark Session](#section2)
- [Load Dataset using Spark DataFrame](#section3)
- [Inspecting the DataFrame](#section4)
- [Dealing with Duplicates](#section5)
- [Dealing with NaN](#section6)
- [Select and Filter a Dataframe](#section7)
- [Running SQL on a DataFrame](#section8)
- [Calculated Columns](#section9)
- [Group By and Aggregation](#section10)
- [Writing DataFrame to File](#section11)

<br>
<hr>

<a id='section1'></a>

## Import all dependencies

In [1]:
import numpy as np
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *             #To define schemas

<a id='section2'></a>

## Create a Spark Session

In [2]:
#Create a spark session
spark = SparkSession.builder.master("local[*]").getOrCreate()

spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/21 15:42:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/21 15:42:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/06/21 15:42:58 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/06/21 15:42:58 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


<a id='section3'></a>

## Load Dataset using Spark DataFrame

In [3]:
os.listdir('input')

['duplicates.csv', 'original.csv']

In [4]:
directory = r'input/original.csv'

#Read csv file using PySpark
df = spark.read.format("csv").option("header", "true").load(directory)

#Show first five lines
df.show(5)

+---+----------+----------+------+-------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|         City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+-------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|       Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|         null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|    Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
+---+----------+----------+------+-------------+--------------------+---------+----------+-----------+
only showing top 5 rows



In [5]:
#Check data types
df.dtypes

[('id', 'string'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('gender', 'string'),
 ('City', 'string'),
 ('JobTitle', 'string'),
 ('Salary', 'string'),
 ('Latitude', 'string'),
 ('Longitude', 'string')]

### Define a new schema (changing datatypes)

In [6]:
#Create a Schema
schema = StructType([
    StructField('id', IntegerType()),
    StructField('first_name', StringType()),
    StructField('last_name', StringType()),
    StructField('gender', StringType()),
    StructField('City', StringType()),
    StructField('JobTitle', StringType()),
    StructField('Salary', StringType()),
    StructField('Latitude', FloatType()),
    StructField('Longitude', FloatType())
])

In [7]:
#Load Schema on dataframe from csv
df = spark.read.csv("input/original.csv", header=True, schema=schema)
df.show(5)

+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
| id|first_name| last_name|gender|         City|            JobTitle|   Salary| Latitude| Longitude|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
|  1|   Melinde| Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|       Bulgan|       Programmer II|$62846.60| 48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|         null|                null|$57576.52|39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|    Mytishchi|            VP Sales|$63863.09|     null| 37.648994|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
only showing top 5 rows



In [8]:
#Datatypes are different now
df.dtypes

[('id', 'int'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('gender', 'string'),
 ('City', 'string'),
 ('JobTitle', 'string'),
 ('Salary', 'string'),
 ('Latitude', 'float'),
 ('Longitude', 'float')]

<br>

[Back to Table of Contents](#tablecontents)

<a id='section4'></a>

## Inspecting the DataFrame

In [9]:
df.show(5)

+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
| id|first_name| last_name|gender|         City|            JobTitle|   Salary| Latitude| Longitude|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
|  1|   Melinde| Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|       Bulgan|       Programmer II|$62846.60| 48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|         null|                null|$57576.52|39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|    Mytishchi|            VP Sales|$63863.09|     null| 37.648994|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
only showing top 5 rows



In [10]:
#See the first row
df.first()

Row(id=1, first_name='Melinde', last_name='Shilburne', gender='Female', City='Nowa Ruda', JobTitle='Assistant Professor', Salary='$57438.18', Latitude=50.57740783691406, Longitude=16.49671745300293)

In [11]:
#As for in pandas, this is the method to see all column names in order
df.columns

['id',
 'first_name',
 'last_name',
 'gender',
 'City',
 'JobTitle',
 'Salary',
 'Latitude',
 'Longitude']

In [12]:
#Count total number of rows in a dataframe
df.count()

1000

In [13]:
#Count distinct values in a dataframe (unique rows) / Quick way to seek for duplicates
df.distinct().count()

1000

In [14]:
#Do it with a function
def spark_duplicates(df):
    all_rows = df.count()
    distinct = df.distinct().count()
    x = all_rows/distinct
    if x == 1:
        print('No Duplicates.')
    else:
        print(f'{all_rows-distinct} rows are duplicate.')
    return

spark_duplicates(df)

No Duplicates.


### Describe method

In [15]:
#Show general stats of a dataframe
df.describe().show()

+-------+-----------------+----------+---------+------+-------------------+-------------------+---------+------------------+-----------------+
|summary|               id|first_name|last_name|gender|               City|           JobTitle|   Salary|          Latitude|        Longitude|
+-------+-----------------+----------+---------+------+-------------------+-------------------+---------+------------------+-----------------+
|  count|             1000|      1000|     1000|  1000|                999|                998|     1000|               999|             1000|
|   mean|            500.5|      null|     null|  null|               null|               null|     null| 25.43151724702484|43.33756460386515|
| stddev|288.8194360957494|      null|     null|  null|               null|               null|     null|24.579082550156635| 69.4206453674681|
|    min|                1|   Abagail|    Abbay|Female|             Abéché|Account Coordinator|$10101.92|         -54.28115|       -123.04196|

In [16]:
#Descriptive Statistics for a numerical column:
df.describe(['Salary']).show()

+-------+---------+
|summary|   Salary|
+-------+---------+
|  count|     1000|
|   mean|     null|
| stddev|     null|
|    min|$10101.92|
|    max|$99948.28|
+-------+---------+



In [17]:
#Do it with a function
def sparkdescribe(df, col):
    return df.describe([col]).show()

for i in ['Latitude', 'Longitude']:
    sparkdescribe(df, i)

+-------+------------------+
|summary|          Latitude|
+-------+------------------+
|  count|               999|
|   mean| 25.43151724702484|
| stddev|24.579082550156635|
|    min|         -54.28115|
|    max|          69.63186|
+-------+------------------+

+-------+-----------------+
|summary|        Longitude|
+-------+-----------------+
|  count|             1000|
|   mean|43.33756460386515|
| stddev| 69.4206453674681|
|    min|       -123.04196|
|    max|        175.28227|
+-------+-----------------+



<br>

[Back to Table of Contents](#tablecontents)

<a id='section5'></a>

## Dealing with Duplicates

In [18]:
#See dataframe with duplicates
df_dup = spark.read.csv("input/duplicates.csv", header=True, schema=schema)
df_dup.show(5)

+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
| id|first_name| last_name|gender|         City|            JobTitle|   Salary| Latitude| Longitude|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
|  1|   Melinde| Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|       Bulgan|       Programmer II|$62846.60| 48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|         null|                null|$57576.52|39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|    Mytishchi|            VP Sales|$63863.09|     null| 37.648994|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
only showing top 5 rows



In [19]:
print(df_dup.distinct().count()) #Print Unique Values
print(df_dup.count())            #Print All Values

49
52


In [20]:
#By using our previous function
spark_duplicates(df_dup)

3 rows are duplicate.


In [21]:
#We know there are duplicate rows, it will be sufficient to use groupby and count.
df_dup.groupBy('id').count().filter('count > 1').show()

+---+-----+
| id|count|
+---+-----+
| 26|    2|
|  1|    2|
| 16|    2|
+---+-----+



In [22]:
#Remove Duplicates
df_no_dupl = df_dup.dropDuplicates()
print(f'Number of unique rows: {df_no_dupl.distinct().count()} on total {df_no_dupl.count()} rows.')

Number of unique rows: 49 on total 49 rows.


<br>

[Back to Table of Contents](#tablecontents)

<a id='section6'></a>

## Dealing with NaN values

In [23]:
#Show where there are null values:
from pyspark.sql.functions import col                               # Call column names in loops
for col_name in df.columns:
    number_null = df.filter(col(col_name).isNull()).count()         # Calculate count of null values for column
    if number_null > 0:
        print(f'{number_null} NaN on {col_name}')                   # Specify column where NaN is found
        df.filter(col(col_name).isNull()).show(5)                   # Print only sliced df with NaN values for specific column

1 NaN on City
+---+----------+---------+------+----+--------+---------+---------+----------+
| id|first_name|last_name|gender|City|JobTitle|   Salary| Latitude| Longitude|
+---+----------+---------+------+----+--------+---------+---------+----------+
|  3|    Alvera| Di Boldi|Female|null|    null|$57576.52|39.994747|116.339775|
+---+----------+---------+------+----+--------+---------+---------+----------+

2 NaN on JobTitle
+---+----------+---------+------+------+--------+---------+---------+----------+
| id|first_name|last_name|gender|  City|JobTitle|   Salary| Latitude| Longitude|
+---+----------+---------+------+------+--------+---------+---------+----------+
|  3|    Alvera| Di Boldi|Female|  null|    null|$57576.52|39.994747|116.339775|
|  7|     Masha|   Divers|Female|Dachun|    null|$25090.87|24.879416|118.930115|
+---+----------+---------+------+------+--------+---------+---------+----------+

1 NaN on Latitude
+---+----------+---------+------+---------+--------+---------+-----

### Case 1: Dropping NaN

In [24]:
# Drop rows where JobTitle isNull
dfx = df.filter(col('JobTitle').isNotNull())
dfx.show(5)

+---+----------+----------+------+---------------+--------------------+---------+---------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary| Latitude| Longitude|
+---+----------+----------+------+---------------+--------------------+---------+---------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60| 48.82316| 103.52182|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|     null| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.426613|-6.1644998|
+---+----------+----------+------+---------------+--------------------+---------+---------+----------+
only showing top 5 rows



### Case 2: Replace NaN with other values

In [25]:
# Replace NULL in city with "Unknown"
dfx = df.withColumn("ClearCity", when(dfx.City.isNull(), "Unknown").otherwise(dfx.City))
dfx.filter(col('ClearCity')=="Unknown").show()

+---+----------+---------+------+----+--------+---------+---------+----------+---------+
| id|first_name|last_name|gender|City|JobTitle|   Salary| Latitude| Longitude|ClearCity|
+---+----------+---------+------+----+--------+---------+---------+----------+---------+
|  3|    Alvera| Di Boldi|Female|null|    null|$57576.52|39.994747|116.339775|  Unknown|
+---+----------+---------+------+----+--------+---------+---------+----------+---------+



<br>

[Back to Table of Contents](#tablecontents)

<a id='section7'></a>

## Select and Filter a Dataframe

- `select` method works on df columns
- `filter` method works on df rows based on a condition

In [26]:
print(df.columns)

['id', 'first_name', 'last_name', 'gender', 'City', 'JobTitle', 'Salary', 'Latitude', 'Longitude']


In [27]:
#Select only first name, last name and salary
df.select("first_name", "last_name", "salary").show(5)

+----------+----------+---------+
|first_name| last_name|   salary|
+----------+----------+---------+
|   Melinde| Shilburne|$57438.18|
|  Kimberly|Von Welden|$62846.60|
|    Alvera|  Di Boldi|$57576.52|
|   Shannon| O'Griffin|$61489.23|
|  Sherwood|   Macieja|$63863.09|
+----------+----------+---------+
only showing top 5 rows



In [28]:
#Filter a dataframe based on first name
df.filter(col("first_name") == 'Masha').show()

+---+----------+---------+------+------+--------+---------+---------+----------+
| id|first_name|last_name|gender|  City|JobTitle|   Salary| Latitude| Longitude|
+---+----------+---------+------+------+--------+---------+---------+----------+
|  7|     Masha|   Divers|Female|Dachun|    null|$25090.87|24.879416|118.930115|
+---+----------+---------+------+------+--------+---------+---------+----------+



In [29]:
#Use a double filter with AND condition
df.filter((col("Latitude") > 24) & (col("Latitude") < 26)).show()

+---+----------+---------+------+----------+--------------------+---------+---------+-----------+
| id|first_name|last_name|gender|      City|            JobTitle|   Salary| Latitude|  Longitude|
+---+----------+---------+------+----------+--------------------+---------+---------+-----------+
|  7|     Masha|   Divers|Female|    Dachun|                null|$25090.87|24.879416| 118.930115|
| 80|     Kassi|  Stoakes|Female|    Pingle|     Web Designer II|$91516.53|24.633362|   110.6433|
|174|     Roman|   Keates|  Male|La Soledad|  Analyst Programmer|$25810.06|24.762173|-104.927895|
|242|     Sunny|   Weddup|Female|     Sanli|   Marketing Manager|$73174.19| 24.84662|  116.74609|
|287|    Alexei|  Harriot|  Male| Shangjing|    Junior Executive|$55711.38|24.390594|  114.52239|
|385|  Harmonie| Portlock|Female|San Isidro|    Graphic Designer|$83451.74|25.723244| -100.36714|
|573|    Ulrica|    Rosas|Female|     Jiupu|           Recruiter|$35165.42|25.748009|  104.51807|
|623|     Emlyn|  Ca

In [30]:
#Use a double filter with OR condition
df.filter((col("Latitude") < 10) | (col("Latitude") > 120)).show()

+---+----------+----------+------+--------------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|                City|            JobTitle|   Salary|  Latitude| Longitude|
+---+----------+----------+------+--------------------+--------------------+---------+----------+----------+
| 10|      Bran|   Trahear|  Male|            Arbeláez|Mechanical System...|$68098.42|  4.272793|-74.416016|
| 11|    Kylynn|   Lockart|Female|            El Cardo|Nuclear Power Eng...|$13604.63|     -5.85| -79.88333|
| 19|  Tiffanie|  Pattison|Female|         Jabungsisir|Senior Financial ...|$91925.08|-7.7232566| 113.46868|
| 35|      Base| Summerlad|  Male|         Changuinola|Business Systems ...|$53137.80|  9.448871| -82.48518|
| 37|     Nicko|     Frays|  Male|              Caxias|      Health Coach I|$99786.40|-4.8654137|   -43.362|
| 41|   Deirdre|Codrington|Female|         Buena Vista| Software Engineer I|$85861.33|-17.458666| -63.66915|
| 48|    Farlee|   

<br>

[Back to Table of Contents](#tablecontents)

<a id='section8'></a>

## Running SQL on a DataFrame

In [31]:
df.show(5)

+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
| id|first_name| last_name|gender|         City|            JobTitle|   Salary| Latitude| Longitude|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
|  1|   Melinde| Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|       Bulgan|       Programmer II|$62846.60| 48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|         null|                null|$57576.52|39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|    Mytishchi|            VP Sales|$63863.09|     null| 37.648994|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
only showing top 5 rows



To run an SQL query on a dataframe, first we create a <b>temporary table</b>.

In [32]:
#Create a Temporary Table
df.createOrReplaceTempView("original")

In [33]:
# Visualize a queried TABLE with WHERE condition
query1 = spark.sql(
    '''
    select * from original
    WHERE City='Bulgan' OR City='Mytishchi'
    ''')
query1.show()

+---+----------+----------+------+---------+-------------+---------+--------+---------+
| id|first_name| last_name|gender|     City|     JobTitle|   Salary|Latitude|Longitude|
+---+----------+----------+------+---------+-------------+---------+--------+---------+
|  2|  Kimberly|Von Welden|Female|   Bulgan|Programmer II|$62846.60|48.82316|103.52182|
|  5|  Sherwood|   Macieja|  Male|Mytishchi|     VP Sales|$63863.09|    null|37.648994|
+---+----------+----------+------+---------+-------------+---------+--------+---------+



In [34]:
# Use a query to concatenate two value columns into a new one
query2 = spark.sql(
    '''
    SELECT concat(first_name, " ", last_name) AS full_name
    FROM original
    WHERE gender='Female'
    ''')
query2.show(5)

+-------------------+
|          full_name|
+-------------------+
|  Melinde Shilburne|
|Kimberly Von Welden|
|    Alvera Di Boldi|
|         Maris Folk|
|       Masha Divers|
+-------------------+
only showing top 5 rows



<br>

[Back to Table of Contents](#tablecontents)

<a id='section9'></a>

## Calculated Columns

### Case 1. Numerical Variables

We can create calculations on numerical variables using operations.

In [35]:
df.show(5)

+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
| id|first_name| last_name|gender|         City|            JobTitle|   Salary| Latitude| Longitude|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
|  1|   Melinde| Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|       Bulgan|       Programmer II|$62846.60| 48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|         null|                null|$57576.52|39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|    Mytishchi|            VP Sales|$63863.09|     null| 37.648994|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
only showing top 5 rows



`Salary` should be a numerical variable but right now it is accounted as string

In [36]:
df.schema['Salary'].dataType

StringType

In [37]:
#Do it with a function
def spark_dtype(column=str, df=df):
    return df.schema[column].dataType

for i in ['id', 'Latitude', 'gender']:
    print(f'Column "{i}" is {spark_dtype(i)}')

Column "id" is IntegerType
Column "Latitude" is FloatType
Column "gender" is StringType


<br>

So first we create a new column:
- we use `substr` method to exclude the <b>$</b> symbol
- we `cast` the new column as float

In [38]:
dfx = df.withColumn('new_salary', df.Salary.substr(2,100).cast('float'))
dfx.show(5)

+---+----------+----------+------+-------------+--------------------+---------+---------+----------+----------+
| id|first_name| last_name|gender|         City|            JobTitle|   Salary| Latitude| Longitude|new_salary|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.577408| 16.496717|  57438.18|
|  2|  Kimberly|Von Welden|Female|       Bulgan|       Programmer II|$62846.60| 48.82316| 103.52182|   62846.6|
|  3|    Alvera|  Di Boldi|Female|         null|                null|$57576.52|39.994747|116.339775|  57576.52|
|  4|   Shannon| O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.504723| 38.130016|  61489.23|
|  5|  Sherwood|   Macieja|  Male|    Mytishchi|            VP Sales|$63863.09|     null| 37.648994|  63863.09|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+----

<br>
Now we can perform our calculation, for example we can calculate the `monthly_salary` based on `new_salary`:

In [39]:
dfx = dfx.withColumn('monthly_salary', dfx.new_salary/12)
dfx.show(5)

+---+----------+----------+------+-------------+--------------------+---------+---------+----------+----------+-----------------+
| id|first_name| last_name|gender|         City|            JobTitle|   Salary| Latitude| Longitude|new_salary|   monthly_salary|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+----------+-----------------+
|  1|   Melinde| Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.577408| 16.496717|  57438.18|4786.514973958333|
|  2|  Kimberly|Von Welden|Female|       Bulgan|       Programmer II|$62846.60| 48.82316| 103.52182|   62846.6|   5237.216796875|
|  3|    Alvera|  Di Boldi|Female|         null|                null|$57576.52|39.994747|116.339775|  57576.52|4798.043294270833|
|  4|   Shannon| O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.504723| 38.130016|  61489.23|  5124.1025390625|
|  5|  Sherwood|   Macieja|  Male|    Mytishchi|            VP Sales|$63863.09|     null| 

In [40]:
# Given the too many decimal numbers we can try to round the column:
from pyspark.sql.functions import round

dfx = dfx.withColumn('RoundedValue', round(col('monthly_salary'), 2))
dfx.show(5)

+---+----------+----------+------+-------------+--------------------+---------+---------+----------+----------+-----------------+------------+
| id|first_name| last_name|gender|         City|            JobTitle|   Salary| Latitude| Longitude|new_salary|   monthly_salary|RoundedValue|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+----------+-----------------+------------+
|  1|   Melinde| Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.577408| 16.496717|  57438.18|4786.514973958333|     4786.51|
|  2|  Kimberly|Von Welden|Female|       Bulgan|       Programmer II|$62846.60| 48.82316| 103.52182|   62846.6|   5237.216796875|     5237.22|
|  3|    Alvera|  Di Boldi|Female|         null|                null|$57576.52|39.994747|116.339775|  57576.52|4798.043294270833|     4798.04|
|  4|   Shannon| O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.504723| 38.130016|  61489.23|  5124.1025390625|      5124.1|

### Case 2. Categorical variables

We can create new columns based on conditions on other variables.

In the case of categorical variables, we can even one-hot encode them.

In [41]:
df.show(5)

+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
| id|first_name| last_name|gender|         City|            JobTitle|   Salary| Latitude| Longitude|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
|  1|   Melinde| Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|       Bulgan|       Programmer II|$62846.60| 48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|         null|                null|$57576.52|39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|    Mytishchi|            VP Sales|$63863.09|     null| 37.648994|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+
only showing top 5 rows



In [42]:
dfz = df.withColumn('is_male', when(df.gender=='Male', 1).otherwise(0))
dfz.show(5)

+---+----------+----------+------+-------------+--------------------+---------+---------+----------+-------+
| id|first_name| last_name|gender|         City|            JobTitle|   Salary| Latitude| Longitude|is_male|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+-------+
|  1|   Melinde| Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.577408| 16.496717|      0|
|  2|  Kimberly|Von Welden|Female|       Bulgan|       Programmer II|$62846.60| 48.82316| 103.52182|      0|
|  3|    Alvera|  Di Boldi|Female|         null|                null|$57576.52|39.994747|116.339775|      0|
|  4|   Shannon| O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.504723| 38.130016|      1|
|  5|  Sherwood|   Macieja|  Male|    Mytishchi|            VP Sales|$63863.09|     null| 37.648994|      1|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+-------+
only showing top 5 

In [43]:
#For one_hot_encoding we need to create a more complex function

In [44]:
from pyspark.sql.functions import collect_set

def spark_onehotEncode(column_name, df=df):
    unique_values = list(df.select(collect_set(column_name).alias('UniqueValues')).first().UniqueValues)
    for feature in unique_values:
        df = df.withColumn(f'is_{feature}', when(col(column_name) == feature, 1).otherwise(0))
    return df

In [45]:
#One-hot encoding gender
dfn = spark_onehotEncode('gender')
dfn.show(5)

+---+----------+----------+------+-------------+--------------------+---------+---------+----------+---------+-------+
| id|first_name| last_name|gender|         City|            JobTitle|   Salary| Latitude| Longitude|is_Female|is_Male|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+---------+-------+
|  1|   Melinde| Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.577408| 16.496717|        1|      0|
|  2|  Kimberly|Von Welden|Female|       Bulgan|       Programmer II|$62846.60| 48.82316| 103.52182|        1|      0|
|  3|    Alvera|  Di Boldi|Female|         null|                null|$57576.52|39.994747|116.339775|        1|      0|
|  4|   Shannon| O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.504723| 38.130016|        0|      1|
|  5|  Sherwood|   Macieja|  Male|    Mytishchi|            VP Sales|$63863.09|     null| 37.648994|        0|      1|
+---+----------+----------+------+-------------+

<br>

[Back to Table of Contents](#tablecontents)

<a id='section10'></a>

## Group By and Aggregation

In [46]:
df1 = dfx.select('id', 'first_name', 'last_name', 'gender', 'Salary', 'RoundedValue')
df1.show(5)

+---+----------+----------+------+---------+------------+
| id|first_name| last_name|gender|   Salary|RoundedValue|
+---+----------+----------+------+---------+------------+
|  1|   Melinde| Shilburne|Female|$57438.18|     4786.51|
|  2|  Kimberly|Von Welden|Female|$62846.60|     5237.22|
|  3|    Alvera|  Di Boldi|Female|$57576.52|     4798.04|
|  4|   Shannon| O'Griffin|  Male|$61489.23|      5124.1|
|  5|  Sherwood|   Macieja|  Male|$63863.09|     5321.92|
+---+----------+----------+------+---------+------------+
only showing top 5 rows



In [47]:
#Group by gender, aggregate by salary
import pyspark.sql.functions as sqlfunc
df1.groupBy('gender').agg(sqlfunc.sum('RoundedValue')).show()

+------+------------------+
|gender| sum(RoundedValue)|
+------+------------------+
|Female|2280376.5799999996|
|  Male|        2343619.63|
+------+------------------+



In [48]:
#Give an alias to the aggregating column
df1.groupBy('gender').agg(sqlfunc.sum('RoundedValue').alias('Sum of Salary')).show()

+------+------------------+
|gender|     Sum of Salary|
+------+------------------+
|Female|2280376.5799999996|
|  Male|        2343619.63|
+------+------------------+



In [49]:
#You can achieve multiple aggregations at once:
df1.groupBy('gender').agg(
    sqlfunc.sum('RoundedValue').alias('Sum of Salary'),
    sqlfunc.avg('RoundedValue').alias('Avg of Salary'),
    sqlfunc.max('RoundedValue').alias('Max of Salary')
).show()

+------+------------------+-----------------+-------------+
|gender|     Sum of Salary|    Avg of Salary|Max of Salary|
+------+------------------+-----------------+-------------+
|Female|2280376.5799999996|4634.911747967479|      8329.02|
|  Male|        2343619.63|4613.424468503937|      8328.58|
+------+------------------+-----------------+-------------+



<br>

[Back to Table of Contents](#tablecontents)

<a id='section11'></a>

## Write a DataFrame to a file

We can save a pyspark dataframe in multiple formats, like:
- csv
- json
- parquet
- xml

In [50]:
dfx.show(5)

+---+----------+----------+------+-------------+--------------------+---------+---------+----------+----------+-----------------+------------+
| id|first_name| last_name|gender|         City|            JobTitle|   Salary| Latitude| Longitude|new_salary|   monthly_salary|RoundedValue|
+---+----------+----------+------+-------------+--------------------+---------+---------+----------+----------+-----------------+------------+
|  1|   Melinde| Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.577408| 16.496717|  57438.18|4786.514973958333|     4786.51|
|  2|  Kimberly|Von Welden|Female|       Bulgan|       Programmer II|$62846.60| 48.82316| 103.52182|   62846.6|   5237.216796875|     5237.22|
|  3|    Alvera|  Di Boldi|Female|         null|                null|$57576.52|39.994747|116.339775|  57576.52|4798.043294270833|     4798.04|
|  4|   Shannon| O'Griffin|  Male|Divnomorskoye|Budget/Accounting...|$61489.23|44.504723| 38.130016|  61489.23|  5124.1025390625|      5124.1|

In [51]:
#Create a new dataframe from few selected data
dfz = dfx.select('id', 'first_name', 'last_name', 'gender', 'JobTitle', 'Latitude', 'Longitude', 'new_salary', 'monthly_salary')

#Handle Null Values (overwriting existing column)
dfz = dfz.withColumn("JobTitle", when(dfz.JobTitle.isNull(), "n/a").otherwise(dfz.JobTitle))

#One-Hot Encode gender
dfz = spark_onehotEncode('gender', dfz)

#Show final dataframe
dfz.show(5)

+---+----------+----------+------+--------------------+---------+----------+----------+-----------------+---------+-------+
| id|first_name| last_name|gender|            JobTitle| Latitude| Longitude|new_salary|   monthly_salary|is_Female|is_Male|
+---+----------+----------+------+--------------------+---------+----------+----------+-----------------+---------+-------+
|  1|   Melinde| Shilburne|Female| Assistant Professor|50.577408| 16.496717|  57438.18|4786.514973958333|        1|      0|
|  2|  Kimberly|Von Welden|Female|       Programmer II| 48.82316| 103.52182|   62846.6|   5237.216796875|        1|      0|
|  3|    Alvera|  Di Boldi|Female|                 n/a|39.994747|116.339775|  57576.52|4798.043294270833|        1|      0|
|  4|   Shannon| O'Griffin|  Male|Budget/Accounting...|44.504723| 38.130016|  61489.23|  5124.1025390625|        0|      1|
|  5|  Sherwood|   Macieja|  Male|            VP Sales|     null| 37.648994|  63863.09|5321.924153645833|        0|      1|
+---+---

In [52]:
#Save dataframe as csv file
try:
    dfz.write.csv('output/csv_output.csv', encoding='UTF-8')
    print('File saved')
except:
    print('Cannot save. Maybe the file already exists.')

File saved


In [53]:
#Save dataframe as json file
try:
    dfz.write.json('output/json_output.json')
    print('File saved')
except:
    print('Cannot save. Maybe the file already exists.')

File saved


In [54]:
#Save dataframe as parquet file
try:
    dfz.write.parquet('output/parquet_output.parquet')
    print('File saved')
except:
    print('Cannot save. Maybe the file already exists.')

File saved


<br>
<b>Note: </b> partitioning and compression options are available for the `write` method (see documentation).

<br>

[Back to Table of Contents](#tablecontents)

In [56]:
print('\n     End of the Notebook :)')


     End of the Notebook :)


23/06/23 12:47:08 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 27004030 ms exceeds timeout 120000 ms
23/06/23 12:47:08 WARN SparkContext: Killing executors is not supported by current scheduler.
